In [1]:
!git clone https://github.com/krasserm/super-resolution.git  #Cloning a Pretrained Model

fatal: Too many arguments.

usage: git clone [<options>] [--] <repo> [<dir>]

    -v, --[no-]verbose    be more verbose
    -q, --[no-]quiet      be more quiet
    --[no-]progress       force progress reporting
    --[no-]reject-shallow don't clone shallow repository
    -n, --no-checkout     don't create a checkout
    --checkout            opposite of --no-checkout
    --[no-]bare           create a bare repository
    --[no-]mirror         create a mirror repository (implies --bare)
    -l, --[no-]local      to clone from a local repository
    --no-hardlinks        don't use local hardlinks, always copy
    --hardlinks           opposite of --no-hardlinks
    -s, --[no-]shared     setup as shared repository
    --[no-]recurse-submodules[=<pathspec>]
                          initialize submodules in the clone
    --[no-]recursive ...  alias of --recurse-submodules
    -j, --[no-]jobs <n>   number of submodules cloned in parallel
    --[no-]template <template-directory>
            

In [2]:
cd /content/super-resolution #Creation of Directory

[WinError 3] The system cannot find the path specified: '/content/super-resolution #Creation of Directory'
c:\Users\Vicky Singh\Downloads\VIDEO-ENHANCEMENT-USING-SINGLE-IMAGE-SUPER-RESOLUTION-master\VIDEO-ENHANCEMENT-USING-SINGLE-IMAGE-SUPER-RESOLUTION-master


c:\Users\Vicky Singh\anaconda3\Lib\site-packages\IPython\core\magics\osm.py:393: UserWarning: using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [ ]:
!tar xvfz /content/weights-srgan.tar.gz #Loading weights

In [ ]:
# Importing all necessary libraries 
import timeit
import cv2 
import os
import numpy as np
from model import resolve_single
from utils import load_image, plot_sample
from model.srgan import generator

# Read the video from specified path 
cam = cv2.VideoCapture("/content/Drama144p_input.3gp") 
fps = cam.get(cv2.CAP_PROP_FPS)
print(fps)


try:
      
    # Creating a folder named data 
    if not os.path.exists('data'): 
        os.makedirs('data') 
  
# if not created then raise error 
except OSError:
    print ('Error: Creating directory of data') 
  
#frames Extraction from video 
currentframe = 0
arr_img = []
while(True): 
      
    # reading from frame 
    ret,frame = cam.read() 
  
    if ret: 
        # if video is still left continue creating images 
        name = './data/frame' + str(currentframe).zfill(3) + '.jpg'
        print ('Creating...' + name) 
  
        # writing the extracted images 
        cv2.imwrite(name, frame) 
  
        # increasing counter so that it will show how many frames are created 
        currentframe += 1
        #storing the path of extracted frames in a list
        arr_img.append(name)
    else: 
        break
#print(arr_img)

start = timeit.default_timer()
model = generator()
model.load_weights('weights/srgan/gan_generator.h5')

#Initialization of an empty list to store the super resolved images
arr_output=[]
print(len(arr_img))
n= len(arr_img)

#Implementation of SRGAN Model in extracted frames
for i in range(n):
  lr = load_image(arr_img[i])
  sr = resolve_single(model, lr)
  #plot_sample(lr, sr)
  
  arr_output.append(sr)
stop = timeit.default_timer()
#print(arr_output)

print("time : ", stop-start)

# Release all space and windows once done 
cam.release() 
cv2.destroyAllWindows()

# **Saving the Super Resolved Frames :-**
The super resolved frames are in numpy array format, so here we will change them in an image format.

In [11]:
#Importing necessary libraries
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
from keras.preprocessing.image import save_img

#Making a directory for storing super resolved frames in image format
os.makedirs("output_images")

#Initialization of an empty list to store the path of Super resolved frames
s_res= []
for j in range(len(arr_output)):
  out_name = '/content/super-resolution/output_images/frame' + str(j).zfill(3) + '.jpg'
  img_pil = array_to_img(arr_output[j])
  img1 = save_img(out_name, img_pil)
  s_res.append(out_name)
  
#print(s_res)

# **Conversion of Super Resolved frames into a video**

In [23]:
import cv2
import numpy as np
for i in range(len(s_res)):
    filename=s_res[i]
    #reading each files
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)

fps = 20       #Put the fps value as your convenience or 
               #Calculate by using (No. of frames)/Video_duration in seconds  

#Creation of output video               
out = cv2.VideoWriter('drama2_output.mp4',cv2.VideoWriter_fourcc(*'DIVX'), fps , size)

#Writing Frames into video
for i in range(len(s_res)):
    out.write(cv2.imread(s_res[i]))
out.release()